## GPT-3 Basics

Sandbox for exploring the GPT-3 API and best practices for prompt engineering.

**Table of Contents:**
1. Prompt organization
2. Ask for specific output structure
3. Inferring
4. Summarizing
5. Transforming
6. Expanding

Resources:
- "ChatGPT Prompt Engineering for Developers" Course by DeepLearning.AI

In [3]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

### 1. Prompt Organization

Let's use a helper function for writing prompts. We'll focus on OpenAI's `gpt-3.5-turbo` model.

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message["content"]

Splitting up the prompt and text for processing can help with organization and readability.

In [9]:
# Wikipedia article on LLMs
text = "A large language model (LLM) is a language model consisting of a neural network with many parameters (typically billions of weights or more), trained on large quantities of unlabeled text using self-supervised learning or semi-supervised learning.[1] LLMs emerged around 2018 and perform well at a wide variety of tasks. This has shifted the focus of natural language processing research away from the previous paradigm of training specialized supervised models for specific tasks.[2]"

In [10]:
prompt = f"""
Summarize the text delimited by triple backticks \ 
into two sentences.
```{text}```
"""
response = get_completion(prompt)
print(response)

A large language model (LLM) is a neural network with billions of weights trained on large amounts of unlabeled text using self-supervised or semi-supervised learning. LLMs have shifted the focus of natural language processing research away from specialized supervised models for specific tasks.


GPT-3 shouldn't care if backslashes or newline characters are used to break up long lines. But there is a risk of newline characters effecting performance when working with LLMs in general.

Let's test newline characters with the same example.

In [13]:
# Wikipedia article on LLMs
text = f"""
A large language model (LLM) is a language model consisting of a neural 
network with many parameters (typically billions of weights or more), 
trained on large quantities of unlabeled text using self-supervised 
learning or semi-supervised learning.[1] LLMs emerged around 2018 and 
perform well at a wide variety of tasks. This has shifted the focus of 
natural language processing research away from the previous paradigm of 
training specialized supervised models for specific tasks.[2]
"""

In [14]:
prompt = f"""
Summarize the text delimited by triple backticks
into one sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

Large language models are neural networks with billions of weights trained on unlabeled text using self-supervised or semi-supervised learning, which have shifted the focus of natural language processing research away from specialized supervised models for specific tasks.


### 2. Ask for Specific Output Structure

LLMs are great at outputting responses in specific data structures such as JSON or HTML for more efficient follow-on processing.

In [1]:
# Fictiticious product review (also generated by GPT-3!)

review = """
I recently purchased an electric sit stand desk and \
I have to say, I'm pretty impressed. The desk is easy \
to adjust with just the touch of a button, and it's \
made a huge difference in my posture and overall \
comfort while working. The desk is also sturdy and \
well-made, so I don't have to worry about it wobbling \
or tipping over. Overall, I'm happy with my purchase \
and would recommend this desk to anyone looking for a \
comfortable and versatile workspace.
"""

In [5]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Sentiment (positive or negative)
- 5 most important keywords discussed in the text

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"item", "sentiment", "keywords" as the keys.

If the information isn't present, use "unknown" \
as the value.

Format the keywords value as a list.

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

{
    "item": "electric sit stand desk",
    "sentiment": "positive",
    "keywords": ["adjust", "posture", "comfort", "sturdy", "versatile"]
}


### 3. Inferring

Inferring is when an LLM takes in text and performs some analysis on it, such as extracting labels, sentiments, summaries, etc.

This is where the power of LLMs shines. In a traditional ML workflow, a new model must be trained and deployed for every task such as identifying labels or analyzing sentiment. In contrast, a LLM can skip the training step and be used across multiple tasks by simply engineering different prompts, drastically accelerating development.

In the example under section "2. Ask for Specific Output Structure", GPT-3 is able to identify the item name, sentiment, and main keywords of a product review without need any additional training steps! 

### 4. Summarizing

Summarizing text is one of the most interesting use cases of LLMs. Imagine you're a large e-commerce website interested in analyzing product reviews. Having a human read through hundreds or thousands of reviews would be a very time-consuming task. An LLM can greatly streamline this process by summarizing each review into a smaller, more efficient summary, enabling a human to read through significantly more reviews in the same amount of time.

In [9]:
# Fictiticious product reviews for a manual coffee grinder (also generated by GPT-3!)
review1 = "This manual coffee grinder is a game-changer! I've been using it for a few weeks now and I can't believe how much better my coffee tastes. The grind is consistent and the process is so satisfying. Plus, it's compact and easy to store. I highly recommend this grinder to any coffee lover out there!"

review2 = "I was really excited to try this manual coffee grinder, but unfortunately it didn't live up to my expectations. The grind wasn't as consistent as I had hoped and it took a lot of effort to grind even a small amount of coffee. I also found it difficult to clean. Overall, I wouldn't recommend this grinder."

review3 = "I absolutely love this manual coffee grinder! It's so easy to use and the grind is perfect every time. I also appreciate that it's made of high-quality materials and feels really sturdy. It's definitely worth the investment if you're serious about your coffee."

review4 = "I have mixed feelings about this manual coffee grinder. On one hand, it's really well-made and looks great on my countertop. On the other hand, it takes a lot of effort to grind even a small amount of coffee and the grind isn't as consistent as I would like. If you're willing to put in the work, it's a decent option, but there are probably better grinders out there."

reviews = [review1, review2, review3, review4]

In [11]:
for review in reviews:
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site.
    
    Also identify the sentiment as either "positive", \
    "negative", or "neutral".
    
    Format your response as a JSON object with \
    "sentiment" and "summary" as the keys.

    Summarize the review below, delimited by triple \
    backticks in at most 40 words. 

    Review: ```{review}```
    """

    response = get_completion(prompt)
    print(response, "\n")

{
    "sentiment": "positive",
    "summary": "This manual coffee grinder is a game-changer! The grind is consistent and the process is satisfying. It's compact and easy to store. Highly recommended for any coffee lover!"
} 

{
    "sentiment": "negative",
    "summary": "Disappointing manual coffee grinder with inconsistent grind and difficult to clean. Not recommended."
} 

{
    "sentiment": "positive",
    "summary": "The manual coffee grinder is easy to use and made of high-quality materials. The grind is perfect every time, making it worth the investment for serious coffee lovers."
} 

{
    "sentiment": "neutral",
    "summary": "Mixed feelings about this manual coffee grinder. Well-made and looks great, but requires a lot of effort to grind and the grind isn't consistent. Decent option if willing to put in the work, but better grinders available."
} 



GPT-3 appears to work best when each API call is small and singular. I first attempted the above as one single API call to process all four reviews rather than using a for loop to process each review in a separate API call. In the former attempt, I would get a connection timeout error.

#### Web links
I've had mixed results with feeding urls to the model. Sometimes it works, sometimes GPT-3 errors and says it "cannot access the internet and browse articles". Below is an example of that such error.

In [6]:
prompt = f"""
Your task is to identify all the sections mentioned \
in an article and summarize the text in them. Return \
your response in the following format for each identified \
section:
- Section:
- Summary:

Each summary should be at least 50 words.

The article is found at the url below, \
delimited by single quotes. 

'https://pravash-techie.medium.com/python-habbits-that-give-away-inexperience-99c00fbeb835'
"""

response = get_completion(prompt)
print(response)

Sorry, as an AI language model, I cannot access the internet and browse articles. Can you please provide the article or a sample text for me to identify the sections and summarize them?


### 5. Transforming

### 6. Expanding